In [ ]:
# Copyright 2024 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Augment Gemeni Output with Vector Embedding from BigQuery

---


<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/generative-ai/blob/main/gemini/use-cases/retrieval-augmented-generation/rag_vector_embedding_in_bigquery.ipynb">
      <img width="32px" src="https://www.gstatic.com/pantheon/images/bigquery/welcome_page/colab-logo.svg" alt="Google Colaboratory logo"><br> Run in Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2FGoogleCloudPlatform%2Fgenerative-ai%2Fmain%2Fgemini%2Fuse-cases%2Fretrieval-augmented-generation%2F%2Frag_vector_embedding_in_bigquery.ipynb">
      <img width="32px" src="https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN" alt="Google Cloud Colab Enterprise logo"><br> Run in Colab Enterprise
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/use-cases/retrieval-augmented-generation/rag_vector_embedding_in_bigquery.ipynb">
      <img width="32px" src="https://upload.wikimedia.org/wikipedia/commons/9/91/Octicons-mark-github.svg" alt="GitHub logo"><br> View on GitHub
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/generative-ai/main/gemini/use-cases/retrieval-augmented-generation/rag_vector_embedding_in_bigquery.ipynb">
      <img src="https://www.gstatic.com/images/branding/gcpiconscolors/vertexai/v1/32px.svg" alt="Vertex AI logo"><br> Open in Vertex AI Workbench
    </a>
  </td>    
</table>

| | | |
|-|-|-|
|Author(s) | [Logan Ramalingam](https://github.com/logan-google)

## Overview

This notebook shows how to use BigQuery to create generate embedding from text in BigQuery table, store them within BigQuery Dataset and use the embeddings to augment the results from LLM in Vector Search.

In this notebook, we create text embedding for publicly available abstracts from [patents data](https://console.cloud.google.com/marketplace/product/google_patents_public_datasets/google-patents-public-data) and use them in our LLM search. Google Patents Public Data, provided by IFI CLAIMS Patent Services, is a worldwide bibliographic and US full-text dataset of patent publications.


```patents-public-data.google_patents_research.publications```

This notebook references the steps mentioned in the [Perform semantic search and retrieval-augmented generation](https://cloud.google.com/bigquery/docs/vector-index-text-search-tutorial)


## Required roles and permissions

To create a connection, you need membership in the following Identity and Access Management (IAM) role:

```roles/bigquery.connectionAdmin```

To grant permissions to the connection's service account, you need the following permission:

```resourcemanager.projects.setIamPolicy```

The IAM permissions needed in this tutorial for the remaining BigQuery operations are included in the following two roles:


*   BigQuery Data Editor (```roles/bigquery.dataEditor```) to create models, tables, and indexes.

*   BigQuery User (```roles/bigquery.user```) to run BigQuery jobs.


## Getting Started

### Install Vertex AI SDK and other required packages


In [ ]:
%pip install --upgrade --user --quiet google-cloud-aiplatform google-cloud-bigquery lxml google-cloud-bigquery-connection

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4.3 MB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.4/233.4 kB 19.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.1/58.1 kB 5.4 MB/s eta 0:00:00
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


### Restart runtime

To use the newly installed packages in this Jupyter runtime, you must restart the runtime. You can do this by running the cell below, which restarts the current kernel.

The restart might take a minute or longer. After its restarted, continue to the next step.

In [ ]:
import sys

if "google.colab" in sys.modules:
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}

<div class="alert alert-block alert-warning">
<b>⚠️ The kernel is going to restart. Please wait until it is finished before continuing to the next step. ⚠️</b>
</div>

### Authenticate your notebook environment (Colab only)

If you are running this notebook on Google Colab, run the cell below to authenticate your environment.

In [ ]:
import sys

if "google.colab" in sys.modules:
    from google.colab import auth

    auth.authenticate_user()

## Set Google Cloud project information and initialize BigQuery Connect

Learn more about [setting up a project and a development environment](https://cloud.google.com/vertex-ai/docs/start/cloud-environment).

In [ ]:
# GCP Project ID
PROJECT_ID = "your-project-id"  # @param {type:"string"}

# BigQuery Dataset for storing embedding and model
DATASET_ID = "bq_vector_embedding"  # @param {type:"string"}

# BigQuery Region
REGION = "US"  # @param {type: "string"}

# BigQuery Connection name
CONN_NAME = "bqml_llm_conn"

# Embedding Model name
EMBEDDING_MODEL_ID = "llm_gecko"  # @param {type:"string"}

# Embedding Table name in BigQuery
EMBEDDING_TABLE_ID = "embeddings"  # @param {type:"string"}

# LLM Model name
LLM_MODEL_ID = "llm_gemini"  # @param {type:"string"}

# LLM Model to use (Use Gemini Pro)
ENDPOINT_TYPE = "gemini-pro"  # @param {type:"string"}

In [ ]:
# Set the project id
! gcloud config set project {PROJECT_ID}

Updated property [core/project].


## Import libraries

Let's start by importing the libraries that we will need for this tutorial

In [ ]:
# enable data_table format
from google.colab import data_table

data_table.enable_dataframe_formatter()
from google.cloud import bigquery
from google.cloud import bigquery_connection_v1 as bq_connection
from google.cloud.exceptions import NotFound

## Setup BigQuery Environment

### Initialize Google BigQuery Client

In [ ]:
client = bigquery.Client(project=PROJECT_ID)

### Wrapper to use BigQuery client to run query and return result

In [ ]:
def run_bq_query(sql: str):
    """
    Input: SQL query, as a string, to execute in BigQuery
    Returns the query results or error, if any
    """
    try:
        query_job = client.query(sql)
        result = query_job.result()
        print(f"JOB ID: {query_job.job_id} STATUS: {query_job.state}")
        return result

    except Exception as e:
        raise Exception(str(e))

### Create BigQuery Dataset to the ML Models

In [ ]:
# Set dataset_id to the ID of the dataset to create.
dataset_id = "{}.{}".format(PROJECT_ID, DATASET_ID)

# Construct a full Dataset object to send to the API.
dataset = bigquery.Dataset(dataset_id)

# Specify the geographic location where the dataset should reside.
dataset.location = "US"

# Send the dataset to the API for creation, with an explicit timeout.
# Raises google.api_core.exceptions.Conflict if the Dataset already
# exists within the project.
try:
    client.get_dataset(dataset_id)  # Make an API request.
    print("Dataset {} already exists".format(dataset_id))
except NotFound:
    dataset = client.create_dataset(dataset, timeout=30)  # Make an API request.
    print("Created dataset {}.{}".format(client.project, dataset.dataset_id))

Dataset logan-playground.bq_vector_embedding already exists


### Create BigQuery Cloud resource connection

You will need to create a [Cloud resource connection](https://cloud.google.com/bigquery/docs/create-cloud-resource-connection) to enable BigQuery to interact with Vertex AI services:

In [ ]:
conn_client = bq_connection.ConnectionServiceClient()
new_conn_parent = f"projects/{PROJECT_ID}/locations/{REGION}"
exists_conn_parent = f"projects/{PROJECT_ID}/locations/{REGION}/connections/{CONN_NAME}"
cloud_resource_properties = bq_connection.CloudResourceProperties({})

# Try to use an existing connection if one already exists. If not, create a new one.
try:
    request = conn_client.get_connection(
        request=bq_connection.GetConnectionRequest(name=exists_conn_parent)
    )
    CONN_SERVICE_ACCOUNT = f"serviceAccount:{request.cloud_resource.service_account_id}"
except Exception:
    connection = bq_connection.types.Connection(
        {"friendly_name": CONN_NAME, "cloud_resource": cloud_resource_properties}
    )
    request = bq_connection.CreateConnectionRequest(
        {
            "parent": new_conn_parent,
            "connection_id": CONN_NAME,
            "connection": connection,
        }
    )
    response = conn_client.create_connection(request)
    CONN_SERVICE_ACCOUNT = (
        f"serviceAccount:{response.cloud_resource.service_account_id}"
    )
print(CONN_SERVICE_ACCOUNT)

serviceAccount:bqcx-843397016286-ims6@gcp-sa-bigquery-condel.iam.gserviceaccount.com


### Set permissions for Service Account
The resource connection service account requires certain project-level permissions which are outlined in the <a href="https://cloud.google.com/bigquery/docs/bigquery-ml-remote-model-tutorial#set_up_access" target="_blank">Vertex AI function documentation</a>.

<br>

**Note:** If you are using Vertex AI Workbench, the service account used by Vertex AI may not have sufficient permissions to add IAM policy bindings.

The [IAM Grant Access](https://cloud.google.com/iam/docs/granting-changing-revoking-access#grant-single-role) page gives instructions on how these policy bindings can be added using Cloud Shell.

In [ ]:
import time

!gcloud projects add-iam-policy-binding {PROJECT_ID} --condition=None --no-user-output-enabled --member={CONN_SERVICE_ACCOUNT} --role='roles/serviceusage.serviceUsageConsumer'
!gcloud projects add-iam-policy-binding {PROJECT_ID} --condition=None --no-user-output-enabled --member={CONN_SERVICE_ACCOUNT} --role='roles/bigquery.connectionUser'
!gcloud projects add-iam-policy-binding {PROJECT_ID} --condition=None --no-user-output-enabled --member={CONN_SERVICE_ACCOUNT} --role='roles/aiplatform.user'
!gcloud services enable bigqueryconnection.googleapis.com
# wait 45 seconds, give IAM updates time to propogate, otherwise, following cells will fail
time.sleep(60)

# Configure Vertex AI EMBEDDINGS Model in BigQuery

## Create the remote model for text embedding generation
Create a remote model that represents a hosted Vertex AI text embedding generation model.

The query takes several seconds to complete, after which the model EMBEDDING_MODEL_ID appears in the DATASET_ID in the Explorer pane.


In [ ]:
sql = f"""CREATE OR REPLACE MODEL
            `{PROJECT_ID}.{DATASET_ID}.{EMBEDDING_MODEL_ID}`
          REMOTE WITH CONNECTION
            `{PROJECT_ID}.{REGION}.{CONN_NAME}`
          OPTIONS (ENDPOINT = 'textembedding-gecko@002');"""
result = run_bq_query(sql)

JOB ID: c3cd6f6c-a960-4d50-9554-8ac99c7254d3 STATUS: DONE


## Generate text embeddings
Generate text embeddings from patent abstracts using the ML.GENERATE_EMBEDDING function, and then write them to a BigQuery table so that they can be searched.

**Note: Query might take upto 10mins to run.**


In [ ]:
sql = f"""
      CREATE OR REPLACE TABLE `{PROJECT_ID}.{DATASET_ID}.{EMBEDDING_TABLE_ID}` AS
      SELECT * FROM ML.GENERATE_TEXT_EMBEDDING(
        MODEL `{PROJECT_ID}.{DATASET_ID}.{EMBEDDING_MODEL_ID}`,
        (
          SELECT *, abstract AS content
          FROM `patents-public-data.google_patents_research.publications`
          WHERE LENGTH(abstract) > 0 AND LENGTH(title) > 0 AND country = 'Singapore'
        )
      )
      WHERE ARRAY_LENGTH(text_embedding) > 0;
      """
result = run_bq_query(sql)

JOB ID: c72f2e54-5927-476c-8947-15f407a588aa STATUS: DONE


## Create Vector index

A vector index is a data structure designed to let the VECTOR_SEARCH function perform a more efficient vector search of embeddings. When VECTOR_SEARCH is able to use a vector index, the function uses the Approximate Nearest Neighbor search technique to help improve search performance, with the trade-off of reducing recall and thus returning more approximate results.

**NOTE: Query might take upto 5mins to run.**


In [ ]:
sql = f"""CREATE OR REPLACE VECTOR INDEX my_index ON `{PROJECT_ID}.{DATASET_ID}.embeddings`(text_embedding) OPTIONS(index_type = 'IVF',
distance_type = 'COSINE',   ivf_options = '{{"num_lists":500}}')"""
result = run_bq_query(sql)

JOB ID: bb9d0084-addf-4da3-a39e-b45f0e88421c STATUS: DONE


### Verify vector index creation

The vector index is populated asynchronously. You can check whether the index is ready to be used by querying the INFORMATION_SCHEMA.VECTOR_INDEXES view and verifying that the coverage_percentage column value is greater than 0 and the last_refresh_time column value isn't NULL.

In [ ]:
#Check vector index creation status, 'coverage_percentage' should be 100
%%bigquery --project {PROJECT_ID}

SELECT table_name, index_name, index_status, coverage_percentage, last_refresh_time, disable_reason
FROM `bq_vector_embedding.INFORMATION_SCHEMA.VECTOR_INDEXES`
WHERE table_name = 'embeddings'

Query is running:   0%|          |

Downloading:   0%|          |

,table_name,index_name,index_status,coverage_percentage,last_refresh_time,disable_reason
0,embeddings,my_index,ACTIVE,100,2024-04-01 21:10:14.472000+00:00,None


## Perform a text similarity search using the vector index

Use the VECTOR_SEARCH function to search for the top 5 relevant patents that match embeddings generated from a text query. The model you use to generate the embeddings in this query must be the same as the one you use to generate the embeddings in the table you are comparing against, otherwise the search results won't be accurate.

In [ ]:
resolvedsql = f"""
  SELECT
    query.query,
    base.content,
    distance
  FROM
    VECTOR_SEARCH( TABLE `{PROJECT_ID}.{DATASET_ID}.{EMBEDDING_TABLE_ID}`,
      'text_embedding',
      (
      SELECT
        text_embedding,
        content AS query
      FROM
        ML.GENERATE_TEXT_EMBEDDING( MODEL `{PROJECT_ID}.{DATASET_ID}.{EMBEDDING_MODEL_ID}`,
          (
          SELECT 'improving password security' AS content))
      ),
      top_k => 5,
      OPTIONS => '{{"fraction_lists_to_search":0.01}}');"""

result = run_bq_query(sql).to_dataframe()
print(result)

JOB ID: 17562073-0f78-4b72-95b1-2c7402f868a4 STATUS: DONE
                         query  \
0  improving password security   
1  improving password security   
2  improving password security   
3  improving password security   
4  improving password security   

                                             content  distance  
0  Methods for improving security in data storage...  0.456121  
1  PASSSWORD MANAGEMENT SYSTEM AND PROCESS There ...  0.473152  
2  METHOD AND APPARATUS FOR UNLOCKING USER INTERF...  0.488237  
3  An active new password entry dialog provides a...  0.488507  
4  An electronic device includes password protect...  0.488999  


# Generate text using embeddings

## Create the remote model for text generation

Create a remote model that represents a hosted Gemeni Model


In [ ]:
sql = f"""
      CREATE OR REPLACE MODEL
        `{PROJECT_ID}.{DATASET_ID}.{LLM_MODEL_ID}`
        REMOTE WITH CONNECTION
          `{PROJECT_ID}.{REGION}.{CONN_NAME}`
        OPTIONS (ENDPOINT = '{ENDPOINT_TYPE}');
      """
result = run_bq_query(sql)


      CREATE OR REPLACE MODEL
        `logan-playground.bq_vector_embedding.llm_gemini`
        REMOTE WITH CONNECTION
          `logan-playground.US.bqml_llm_conn`
        OPTIONS (ENDPOINT = 'gemini-pro');
      
JOB ID: 81ebf7b5-f293-43fb-8d6e-1b1cdafdd80b STATUS: DONE


## Generate text augmented by vector search results

Feed the search results as prompts to generate text with the ML.GENERATE_TEXT function


In [ ]:
sql = f"""SELECT ml_generate_text_llm_result AS generated, prompt
FROM ML.GENERATE_TEXT(
  MODEL `{PROJECT_ID}.{DATASET_ID}.{LLM_MODEL_ID}`,
  (
    SELECT CONCAT(
      'Propose some project ideas to improve user password security using the context below. Add the patent title and url to each idea: ',
      STRING_AGG(
        FORMAT("patent title: %s, patent abstract: %s", base.title, base.abstract))
      ) AS prompt,
    FROM VECTOR_SEARCH(
      TABLE `{PROJECT_ID}.{DATASET_ID}.{EMBEDDING_TABLE_ID}`, 'text_embedding',
      (
        SELECT ml_generate_embedding_result, content AS query
        FROM ML.GENERATE_EMBEDDING(
          MODEL `{PROJECT_ID}.{DATASET_ID}.{EMBEDDING_MODEL_ID}`,
         (SELECT 'improving password security' AS content)
        )
      ),
    top_k => 5, options => '{{"fraction_lists_to_search": 0.01}}')
  ),
  STRUCT(600 AS max_output_tokens, TRUE AS flatten_json_output));"""

query_job = client.query(sql)
rows = query_job.result()

for row in rows:
    print(row[0])

**Project Idea 1:**

**Patent Title:** Data Storage Device Security Method and Apparatus
**URL:** https://patents.google.com/patent/US20060190331A1/en

**Project Description:**
Develop a password management system that utilizes a synchronization method to encrypt and change passwords at each transmission from host to data storage device. This enhances password security by preventing unauthorized access to stored data.

**Project Idea 2:**

**Patent Title:** Passsword Management System and Process
**URL:** https://patents.google.com/patent/US20060287933A1/en

**Project Description:**
Create a password management system that updates passwords based on functional account data and scheduling criteria. This allows for automated password updates, reducing the risk of password compromise due to outdated credentials.

**Project Idea 3:**

**Patent Title:** Method and Apparatus for Unlocking User Interface
**URL:** https://patents.google.com/patent/CN105343265A/en

**Project Description:**
Deve

## Cleaning up

Clean up resources created in this notebook. If no resources are created in this notebook, remove this section

In [ ]:
# Delete Vector Index
sql = f"""DROP VECTOR INDEX my_index ON `{PROJECT_ID}.{DATASET_ID}.{EMBEDDING_TABLE_ID}`"""
result = run_bq_query(sql)

# Delete Gemeni Model
sql = f"""DROP MODEL `{PROJECT_ID}.{DATASET_ID}.{LLM_MODEL_ID}`"""
result = run_bq_query(sql)

# Delete Embedding Model
sql = f"""DROP MODEL `{PROJECT_ID}.{DATASET_ID}.{EMBEDDING_MODEL_ID}`"""
result = run_bq_query(sql)

# Delete Embedding Table
sql = f"""DROP TABLE `{PROJECT_ID}.{DATASET_ID}.{EMBEDDING_TABLE_ID}`"""
result = run_bq_query(sql)

# Delete BigQuery Connection
request = bq_connection.DeleteConnectionRequest({"name": exists_conn_parent})
response = conn_client.delete_connection(request)

# Delete Dataset
client.delete_dataset(dataset, delete_contents=True, not_found_ok=True)

# Close BigQuery Connection
client.close()